# Задание 1 (10 баллов)

Обучите языковую модель с помощью GRPO на вот этом датасете - `notbadai/python_functions_reasoning`
Он чем-то похож на датасет из семинара, но вместо математических задач нужно написать код на питоне.
Измените system prompt под эту задачу. Напишите аналогичные reward функции - одна на проверку формата (markdown wrappers ```python... ``` вокруг кода в конце), и на проверку что код работает (можете использовать import ast; ast.parse(string) для проверки)
Если модель сначала не может отвечать в нужно формате, то сделайте цикл SFT обучения на небольшом куске датасета.
GRPO в колабе может работать долго, поэтому не старайтесь пропустить весь датасет. Но обучайте хотя бы на 100 промптах (по несколько генераций на каждый промпт).
Оцените разницу в качестве на небольшом сабсете, прогнав изначальную модель, модель после SFT (если она есть) и GRPO модель. В качестве метрики используйте reward функции.
Можете взять любую языковую модель любого размера. Можете дообучать всю модель или же только адаптера поверх квантизированной модели.


In [1]:
%pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-y4o7omue
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-y4o7omue
  Resolved https://github.com/huggingface/transformers to commit 2932f318a20d9e54cc7aea052e040164d85de7d6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11588373 sha256=0858f319a6c369daeee23e1859284143e4b99986c6f749df35b4688d73dfc3e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-h5_dg4zd/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [2]:
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from datasets import load_dataset
from trl import GRPOConfig, GRPOTrainer


from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from trl import DPOTrainer
from datasets import load_dataset
from tqdm.notebook import tqdm
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training



from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

In [5]:
system_prompt = """
Your task is to write Python code. Wrap the answer in ```python\n*answer*\n```. Supplement the code with comments, explaining what each command does.
"""

def chatml_format(example):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": example['prompt']}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    return {
        "text": prompt,
    }

In [6]:
! pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.6 MB/s eta 0:00:00


In [7]:
model_name = "Qwen/Qwen2-0.5B-Instruct"
device = 'cuda'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [8]:
# разделяем датасет на трейн и тест
ds = load_dataset("notbadai/python_functions_reasoning", "default")['train'].train_test_split(test_size=0.2, seed=42)
ds

README.md:   0%|          | 0.00/668 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/414M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/206299 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'reasoning', 'answer', 'source'],
        num_rows: 165039
    })
    test: Dataset({
        features: ['prompt', 'reasoning', 'answer', 'source'],
        num_rows: 41260
    })
})

In [9]:
sample_count = 200

train_dataset = ds['train'].shuffle(seed=42).select(range(sample_count)).map(chatml_format)
test_dataset = ds['test'].shuffle(seed=42).select(range(sample_count)).map(chatml_format)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [10]:
train_dataset[0]

{'prompt': '```python\ndef smallest_divisor_larger_than(num, start):\n    """Write a function to find the smallest divisor of a given number that is larger than a specified starting value. If no such divisor exists, the function should return the number itself.\n    """\n```',
 'reasoning': 'To implement the function `smallest_divisor_larger_than`, we need to follow these steps:\n\n1. **Understand the Inputs and Outputs:**\n   - The function takes two parameters:\n     - `num`: an integer representing the number for which we want to find a divisor.\n     - `start`: an integer representing the starting value for finding the divisor.\n   - The function should return the smallest divisor of `num` that is greater than `start`.\n   - If no such divisor exists, it should return `num` itself.\n\n2. **Find Divisors:**\n   - A divisor of `num` is an integer that divides `num` without leaving a remainder.\n   - We need to check numbers starting from `start + 1` upwards to find the smallest divis

In [11]:
import re
def format_accuracy_func(completions, **kwargs):
    rewards = []
    for response in completions:
        if re.search('```python\n(.*?)\n```', response, re.DOTALL):
            rewards.append(1.0)

        else:
            rewards.append(0.0)

    return rewards

def answer_accuracy_func(completions, **kwargs):
    rewards = []
    for response in completions:
        match_c = re.search('```python\n(.*?)\n```', response, re.DOTALL)

        rewards.append(1.0) if match_c is not None else rewards.append(0.0)

    return rewards

In [12]:
batch = train_dataset[2:3]

In [13]:
input_ids = tokenizer.batch_encode_plus(batch['prompt'], return_tensors='pt', padding=True)

output = model.generate(
    input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
    max_new_tokens=400, do_sample=True, temperature=1.5, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
)

In [14]:
completions = tokenizer.batch_decode(output, skip_special_tokens=False)

In [15]:
completions[0]

'Write a function to determine, for each string in a list of queries, how many strings in a list of words have a greater frequency of their smallest character. The frequency of a character is defined as the number of times it appears in the string. The function should return a dictionary where keys are the queries and values are dictionaries containing the count of how often each query appears in the corresponding word in the word list.\n\nFunction Signature:\n```python\ndef freq_words_by_characters(words_list: List[str], queries: List[List[int]]) -> Dict[Tuple[str, str], Dict[str, int]]:\n    pass  # Implementing your solution here\n```\n\nExample:\n```python\nfreq_words = freq_words_by_characters([["abcde"], ["abce", "bc", "abcd"], ["abc"], ["a"]], [[10, 5], [5, 2]])\n```\nOutput:\n```yaml\n{\'queries_abcdef\': {\'a\': 2}, \'queries_bcd\': {\'c\': 4}}\n```\nExplanation:\nThe first `words_list` has three queries, and these queries can be categorized based on their respective character

In [16]:
format_accuracy_func(completions)

[1.0]

In [17]:
answer_accuracy_func(completions)

[1.0]

In [18]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear"
)

In [19]:
training_args = GRPOConfig(output_dir="Qwen/Qwen2-0.5B-Instruct-math-sft-grpo",
                           logging_steps=10,
                           report_to="none",
                           num_generations=8,
                           num_train_epochs=1,
                           temperature=1.5,
                           label_names=["labels"])

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[answer_accuracy_func, format_accuracy_func],
    args=training_args,
    train_dataset=train_dataset,
    peft_config=peft_config

)

In [20]:
trainer.train()

Step,Training Loss
10,-0.032600
20,-0.000600
30,-0.000000
40,-0.006400
50,0.001300
60,-0.013000
70,0.002000
80,-0.003200
90,-0.010800
100,0.014400


TrainOutput(global_step=200, training_loss=-0.0034510158790419607, metrics={'train_runtime': 4011.6286, 'train_samples_per_second': 0.05, 'train_steps_per_second': 0.05, 'total_flos': 0.0, 'train_loss': -0.0034510158790419607})

In [21]:
trainer.save_model('Qwen/Qwen2-0.5B-Instruct-math-sft-grpo')

In [22]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct-math-sft-grpo")
tokenizer.pad_token = tokenizer.eos_token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

In [27]:
response = []

batch_size = 2
for i in tqdm(range(0, 100, batch_size)):
    batch = test_dataset[i:i+batch_size]
    input_ids = tokenizer.batch_encode_plus(batch['prompt'].tolist(), return_tensors='pt', padding=True)

    output = model.generate(
        input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
        max_new_tokens=200, do_sample=False, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
    )[:,input_ids['input_ids'].shape[-1]:]
    response.extend(tokenizer.batch_decode(output, skip_special_tokens=True))

  0%|          | 0/50 [00:00<?, ?it/s]

In [36]:
response

[' The program should take an input array as its parameter and return the maximum product of any contiguous subarray within it.\n\nExample:\nInput: [2, 3, -2, 4]\nOutput: 6\n\nConstraints:\n- The length of the input array will be between 1 and 500.\n- Each element in the array will be between -10^9 and 10^9.\n\nNote:\n- You may assume that the input array contains at least one positive integer.\n- The output value should be returned as an integer.\n\n```python\ndef max_product_subarray(arr):\n    """\n    Finds the maximum product of any contiguous subarray within the given array.\n    \n    :param arr: List[int] - A list of integers.\n    :return: int - The maximum product of any contiguous subarray.\n    """\n    if not arr:\n        return 0\n    \n    max_product = current_max = global_max = arr[0]\n    \n    for i',
 ' The function should have a time complexity of O(n), where n is the length of the input string.\n\nExample:\nInput: s = "abcde", p = "bce"\nOutput: "bc"\n\nInput: s 

In [24]:
model_ref = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct-math-sft-grpo", device_map='cuda', torch_dtype=torch.bfloat16)

In [25]:
device = model_ref.device
response_ref = []
test_dataset.set_format("pandas")
batch_size = 2
for i in tqdm(range(0, 100, batch_size)):
    batch = test_dataset[i:i+batch_size]
    input_ids = tokenizer.batch_encode_plus(batch['prompt'].tolist(), return_tensors='pt', padding=True)

    output = model_ref.generate(
        input_ids['input_ids'].to(device), attention_mask=input_ids['attention_mask'].to(device),
        max_new_tokens=200, do_sample=False, pad_token_id=tokenizer.eos_token_id #**gen_kwargs
    )[:,input_ids['input_ids'].shape[-1]:]
    response_ref.extend(tokenizer.batch_decode(output, skip_special_tokens=True))


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:641: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:658: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [35]:
response_ref

[' The program should take an input array as its parameter and return the maximum product of any contiguous subarray within it.\n\nFor example, given the input array [2, 3, -2, 4], the program should output 6 (as the maximum product is obtained by multiplying 4 and -2).\n\nYou are not allowed to use any built-in functions or libraries for this task.\n```python\ndef max_product_subarray(arr):\n    """\n    Finds the maximum product of any contiguous subarray within the given array.\n    \n    :param arr: List[int] - A list of integers.\n    :return: int - The maximum product of any contiguous subarray.\n    """\n    if len(arr) == 0:\n        return 0\n    \n    # Initialize variables to store the maximum product so far and the minimum product found so far.\n    max_product = min_product = arr[0]\n    \n    for num in arr[1:]:\n        # Update the maximum product if the current',
 ' The function should have a time complexity of O(n), where n is the length of the input string.\n\nExampl

In [28]:
result_df = pd.DataFrame({'prompt': test_dataset['prompt'][:100], 'answer': test_dataset['answer'][:100], 'response (before)': response_ref, 'response (after)': response})

In [38]:
def parse_answer(text):
    m = re.search('python\n(.*?)\n', text)
    if m is not None:
        return m.group(1)
    else:
        return ""

In [34]:
result_df

,prompt,answer,response (before),response (after),answer_before,answer_after,answer_correct
0,Write a Python program to find the maximum pro...,```python\ndef max_product_subarray(arr):\n ...,The program should take an input array as its...,The program should take an input array as its...,,,
1,Write a function to find the minimum window su...,```python\nfrom collections import Counter\n\n...,The function should have a time complexity of...,The function should have a time complexity of...,,,
2,```python\nfrom collections import deque\n\n\n...,```python\n if end not in bank:\n re...,python\n\n# Import necessary packages at the b...,python\n\n# Import necessary packages at the b...,,,
3,Write a function to find the maximum subarray ...,```python\ndef max_subarray_min_product(nums):...,"\n\nFor example, given an input array \(\[2,3...","\n\nFor example, given an input array \(\[2,3...",,,
4,Write a function to compute the sorted squares...,```python\ndef sorted_squares(nums):\n # Sq...,"For example, given the input [1, 2, 3], the o...","For example, given the input [1, 2, 3], the o...",,,
...,...,...,...,...,...,...,...
95,"```python\ndef find_paths(board):\n """"""Writ...","```python\n m, n = len(board), len(board[0]...",python\n\n# Importing necessary package for de...,python\n\n# Importing necessary package for de...,,,
96,"```python\ndef merge_lists(list_1, list_2):\n ...",```python\n return list_1 + list_2\n```</s>,python\n\n# Importing necessary package for de...,python\n\n# Importing necessary package for de...,,,return list_1 + list_2
97,```python\nfrom typing import List\n\n\ndef od...,```python\n if n <= 0:\n return []\n...,"python\n\n# Example usage:\nif __name__ == ""__...",python\nfrom typing import List\n\ndef odd_num...,,,
98,"```python\ndef all_substrings(s):\n """"""Writ...",```python\n result = []\n length = len(s...,python\n\n# Import necessary packages (if any ...,python\n\n# Import necessary packages (if any ...,,,


In [39]:
result_df['answer_before'] = result_df["response (before)"].apply(parse_answer)
result_df['answer_after'] = result_df["response (after)"].apply(parse_answer)
result_df['answer_correct'] = result_df["answer"].apply(parse_answer)

In [40]:
result_df[result_df['answer_before'] != result_df['answer_after']][['prompt', 'answer_before', 'answer_after', 'answer_correct']]

,prompt,answer_before,answer_after,answer_correct
3,Write a function to find the maximum subarray ...,from typing import List,,def max_subarray_min_product(nums):
6,Write a function to encode a categorical varia...,,def categorize_status(status):,def encode_education_status(education_status):
9,Write a function to convert a given list of nu...,def square_dict(numbers):,def square_and_cube(numbers):,def number_to_square_or_cube_dict(numbers):
15,Write a function to convert a given positive i...,def excel_column_name_converter(num):,def convert_to_excel_column_name(num):,def convert_to_excel_column(n: int) -> str:
24,Write a function to generate all possible lett...,,def generate_combinations(digits):,from typing import List
29,Write a function to calculate the total durati...,from typing import List,import datetime,"def find_poisoned_duration(time_series, durati..."
39,```python\nfrom typing import List\n\n\ndef is...,,from typing import List,perimeter = 0
47,```python\ndef count_uppercase_words(sentence)...,,import re,# Split the sentence into words
60,Write a function to find the intersection of t...,from collections import Counter,from typing import List,"def intersect_lists(listA: list, listB: list) ..."
62,Write a function that separates the odd number...,,def separate_odd_even(arr):,def separate_odd_even(nums):


In [41]:
(result_df['answer_before'] == result_df['answer_correct']).sum()/len(result_df)

np.float64(0.02)

In [42]:
(result_df['answer_after'] == result_df['answer_correct']).sum()/len(result_df)

np.float64(0.02)